In [ ]:
'''Import basic modules.'''
import pandas as pd
import numpy as np


'''Customize visualization
Seaborn and matplotlib visualization.'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline
!pip install rake-nltk

'''Plotly visualization .'''
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
py.init_notebook_mode(connected = True) # Required to use plotly offline in jupyter notebook

import cufflinks as cf #importing plotly and cufflinks in offline mode  
import plotly.offline  
cf.go_offline()  
cf.set_config_file(offline=False, world_readable=True)

'''Display markdown formatted output like bold, italic bold etc.'''
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

In [ ]:
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data.head()

In [ ]:
from wordcloud import WordCloud
# print(' '.join(data['title']))
plt.rcParams['figure.figsize'] = (13, 13)

plt.imshow(WordCloud(max_words=50, width=1000, height=1000).generate(' '.join(data['title'])))

In [ ]:
# data['type'].value_counts().reset_index().plot.pie(autopct="%.1f%%",subplots=True)
td=data['type'].value_counts().reset_index()
# print(td)
plt.pie(x=td['type'],autopct="%.1f%%", labels=td['index'])

# py.iplot([pie_plot(data['type'].value_counts(), ['cyan', 'gold'], 'Content Type')])

In [ ]:
from rake_nltk import Rake
new_df = data[['title','director','cast','listed_in','description']]
new_df['Key_words'] = ""

for index, row in new_df.iterrows():
    description = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(description)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
new_df.drop(columns = ['description'], inplace = True)

In [ ]:
new_df.head()

In [ ]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
# new_df.dropna(inplace=True)

# new_df['cast'] = new_df['cast'].map(lambda x: x.split(',')[:3])

# # putting the genres in a list of words
# new_df['listed_in'] = new_df['listed_in'].map(lambda x: x.lower().split(','))

# new_df['director'] = new_df['director'].map(lambda x: x.split(' '))

# # merging together first and last name for each actor and director, so it's considered as one word 
# # and there is no mix up between people sharing a first name
# for index, row in new_df.iterrows():
#     row['cast'] = [x.lower().replace(' ','') for x in row['cast']]
#     row['director'] = ''.join(row['director']).lower()

In [ ]:
new_df.dropna(inplace=True)
new_df['cast'] = new_df['cast'].map(lambda x : x.lower().split(',')[:3])
new_df['cast'] = new_df['cast'].map(lambda y : [x.replace(' ', '') for x in y])



In [ ]:
new_df['director'] = new_df['director'].map(lambda x : x.replace(' ', '') )
new_df['listed_in'] = new_df['listed_in'].map(lambda x : x.lower().split(',')[:3])

new_df['listed_in'] = new_df['listed_in'].map(lambda y : [x.replace(' ', '') for x in y])

new_df.head()

In [ ]:

new_df['bag'] = ''
for i, row in new_df.iterrows():
    words = ' '
    for col in new_df.columns:
        if col != 'director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]
    row['bag']=words
new_df.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
new_df.set_index('title', inplace = True)

count = CountVectorizer()
cnt_mat = count.fit_transform(new_df['bag'])
indices = pd.Series(new_df.index)
indices[:5]

In [ ]:
cnt_mat.shape

In [ ]:
cos_sim = cosine_similarity(cnt_mat, cnt_mat)
cos_sim.shape


In [ ]:
def recommendations(Title, cosine_sim = cos_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == Title].index[0]
    print(indices[indices == Title].index[0])
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(new_df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Rocky')